# Introduction

This tutorial is about the transformation packages `LocalCoordinateSystem` class which describes the orientation of a local cartesian coordinate system towards the reference coordinate system with: origin = (0, 0, 0), e_x = (1, 0, 0), e_y = (0, 1, 0), e_z = (0, 0, 1)). 

The packages required in this tutorial are:

In [1]:
%matplotlib widget

In [2]:
# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

#interactive plots
import ipywidgets as widgets
from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import numpy as np

import mypackage.visualization as vs
import mypackage.transformations as tf

# Construction

The constructor of the `LocalCoordinateSystem` class takes 2 parameters, the `basis` and the `origin`. `basis` is a 3x3 matrix, were each column is one basis vector of the local coordinate system we want to define. The basis vectors need to be orthogonal, otherwise an exception is raised. Note that every pure rotation and reflection matrix is a valid orthogonal base. `origin` is the position of the local coordinate systems origin inside the base coordinate system. Both parameters have a default value, which is equal to the reference coordinate systems value. So if no parameter is specified, we get the reference coordinate system:

In [3]:
cs_ref = tf.LocalCoordinateSystem()

We create some coordinate systems and visualize them using the `visualization` package. The coordinate axes are colored as follows: 
- x = red
- y = green
- z = blue

In [4]:
# create a translated coordinate system 
cs_01 = tf.LocalCoordinateSystem(origin = [2, 4, -1])

# create a rotated coordinate system using a rotation matrix as basis
rotation_matrix = tf.rotation_matrix_z(np.pi/3)
cs_02 = tf.LocalCoordinateSystem(basis= rotation_matrix, origin = [0, 0, 3])

# create 3d plot
fig = plt.figure()
ax = fig.gca(projection='3d')
fig.canvas.layout.height="500px"
fig.canvas.layout.width="500px"

vs.plot_coordinate_system(cs_ref, ax, color='r', label="reference system")
vs.plot_coordinate_system(cs_01, ax, color='g', label="system 1")
vs.plot_coordinate_system(cs_02, ax, color='b', label="system 2")
vs.set_axes_equal(ax)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Apart from the class constructor, there are some factory functions implemented to create a coordinate system. The `construct_from_orientation` provides the same functionality as the class constructor. The `construct_construct_from_xyz` takes 3 basis vectors instead of a matrix. `construct_from_xy_and_orientation`, `construct_from_xz_and_orientation` and `construct_from_yz_and_orientation` create a coordinate system with 2 basis vectors and a bool which speciefies if the coordinate system should have a positive or negative orientation. Here are some examples:

In [5]:
# coordinate system using 3 basis vectors
e_x = [1, 2, 0]
e_y = [-2, 1, 0]
e_z = [0, 0, 5]
cs_03 = tf.LocalCoordinateSystem.construct_from_xyz(e_x, e_y, e_z, origin = [1, 1, 0])

# create a negatively oriented coordinate system with 2 vectors
cs_04 = tf.LocalCoordinateSystem.construct_from_yz_and_orientation(e_y,e_z,positive_orientation=False, origin=[1,1,2])

# create 3d plot
fig = plt.figure()
ax = fig.gca(projection='3d')
fig.canvas.layout.height="500px"
fig.canvas.layout.width="500px"

vs.plot_coordinate_system(cs_ref, ax, color='r', label="reference system")
vs.plot_coordinate_system(cs_03, ax, color='g', label="system 3")
vs.plot_coordinate_system(cs_04, ax, color='b', label="system 4")
vs.set_axes_equal(ax)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As you can see, the y and z axis of system 3 and 4 point into the same direction, since we used the same basis vectors. The automatically determined x axis of system 4 points into the opposite direction, since we wanted a system with negative orientation.

# Coordinate transformations

It is quite common that there exist a chain or tree like dependency between coordinate systems. We might have a moving object with a local coordinate system which describes its position and orientation towards a fixed reference coordinate system. This object can have another object attached to it, with its position and orientation given in relation to its parent objects coordinate system. If we now want the attached object coordintae system in relation to the reference coordinate system, we have to perform a coordinate transformation. 

The reference coordinate system can easily be changed using the `+` and `-` operators of the `LocalCoordinateSystem` class. Let's say we have 3 coordinate systems `cs_ref`, `cs_parent` and `cs_child`, with `cs_ref` being the reference coordinate system of `cs_parent` and `cs_parent` being the reference coordinate system of `cs_child`. Now we want to get a coordinate system `cs_child_ref` which is equivalent to `cs_child` but in relation to `cs_ref`. This is achieved with:

~~~ python
cs_child_ref = cs_child + cs_parent
~~~

It is important to remember that this operation is in general not cummutative, since it involves matrix multiplication which is also not commutative. During coordinate system addition, the local system that should be transformed into another coordinate system is always located to the left of the `+` or `-` operator. However, coordinate system addition is associative so that

~~~ python
cs_child_child + (cs_child + cs_parent) == (cs_child_child + cs_child) + cs_parent
~~~

So far we have seen how to transform a childs coordinate system to the reference coordinate system of its parent using the `+` operator. The `-` operator performs the opposite operation if the transformed and the target coordinate system have a **common reference coordinate system**. The following instruction cancels the previously applied addition:

~~~ python
cs_child = cs_child_ref - cs_parent
~~~

`cs_child_ref` and `cs_parent` have both `cs_ref` as reference coordinate system.

As for addition, coordinate system substraction is not commutative, but in contrast to addition, it is also not associative. The reason for this is that transformed and target system must have the same reference system. To understand why this excludes associativity, look at the following example:

~~~ python
cs_01 - cs_parent - cs_child
~~~

`cs_child` has `cs_parent` as reference system and `cs_01` and `cs_parent` share a common reference system. This operation should give us `cs_01` in  relation to `cs_child`. In python it is equivalent to:

~~~ python
(cs_01 - cs_parent) - cs_child      # added paranthesis
~~~

Since `cs_01` and `cs_parent` share a common reference system, `cs_01 - cs_parent` gives us `cs_01` in relation to `cs_parent`. `cs_child` also has `cs_parent` as reference system. So we can substract `cs_child` subsequently from `cs_01 - cs_parent` to get `cs_parent` in relation to `cs_child`.

Now look at the following case:

~~~ python
cs_01 - (cs_parent - cs_child)
~~~

`cs_parent - cs_child` gives us `cs_parent` in relation to `cs_child`. Therefore the difference `cs_parent - cs_child` and `cs_01` don't share a common reference coordinate system and can't be substracted from each other. 





In [6]:
def coordinate_system_addition(parent_orientation, parent_origin, child_orientation, child_origin):   
    cs_parent = tf.LocalCoordinateSystem(basis=parent_orientation, origin=parent_origin)
    cs_child = tf.LocalCoordinateSystem(basis=child_orientation, origin=child_origin)
    cs_child_ref = cs_child + cs_parent
    
    return [cs_parent, cs_child, cs_child_ref]

Now just execute the following code cells:

In [7]:
###
###
### NO NEED TO UNDERSTAND THIS CODE, IT CONTAINS NOTHING RELEVANT TO UNDERSTAND THIS TUTORIAL - JUST EXECUTE IT
###
###

disp_lim = 3

# create output widget that will hold the figure
out = widgets.Output(layout={'border': '2px solid black'})

# create figure inside output widget
with out:
    fig = plt.figure()
    fig.canvas.layout.height="900px"
    fig.canvas.layout.width="900px"
    gs = fig.add_gridspec(3, 2)
    ax_0 = fig.add_subplot(gs[0, 0], projection='3d')
    ax_1 = fig.add_subplot(gs[0, 1], projection='3d')
    ax_2 = fig.add_subplot(gs[1:, 0:], projection='3d')
    
def setup_axes(axes, limit, title = ""):
    axes.set_xlim([-limit, limit])
    axes.set_ylim([-limit, limit])
    axes.set_zlim([-limit, limit])
    axes.set_xlabel("x")
    axes.set_ylabel("y")
    axes.set_zlabel("z")
    axes.set_title(title)
    axes.legend(loc="lower left")
    

    
def update_output(p_x, p_y, p_z, p_rx, p_ry, p_rz, c_x, c_y, c_z, c_rx, c_ry, c_rz):
    
    p_rx = p_rx / 180 * np.pi
    p_ry = p_ry / 180 * np.pi
    p_rz = p_rz / 180 * np.pi
    c_rx = c_rx / 180 * np.pi
    c_ry = c_ry / 180 * np.pi
    c_rz = c_rz / 180 * np.pi
    
    parent_orientation = np.matmul(tf.rotation_matrix_z(p_rz),
                                   np.matmul(tf.rotation_matrix_y(p_ry), tf.rotation_matrix_x(p_rx)))    
    parent_origin=[p_x, p_y, p_z]
    child_orientation = np.matmul(tf.rotation_matrix_z(c_rz),
                                  np.matmul(tf.rotation_matrix_y(c_ry), tf.rotation_matrix_x(c_rx)))    
    child_origin=[c_x, c_y, c_z]
    
    cs_ref = tf.LocalCoordinateSystem()
    [cs_parent, cs_child, cs_child_ref] = coordinate_system_addition(parent_orientation, parent_origin, child_orientation, child_origin)
            
    origin_cr = cs_child_ref.origin
    cr_x = origin_cr[0]
    cr_y = origin_cr[1]
    cr_z = origin_cr[2]
    
    ax_0.clear()
    vs.plot_coordinate_system(cs_ref, ax_0, color='r', label="reference")
    vs.plot_coordinate_system(cs_parent, ax_0, color='g', label="parent")
    ax_0.plot([0, p_x], [0, p_y], [0, p_z], 'c--', label="ref -> parent")
    setup_axes(ax_0, disp_lim, "'parent' in reference coordinate system")
    
    
    ax_1.clear()
    vs.plot_coordinate_system(cs_ref, ax_1, color='g', label="parent")
    vs.plot_coordinate_system(cs_child, ax_1, color='y', label="child")
    ax_1.plot([0, c_x], [0, c_y], [0, c_z], 'm--', label="parent -> child")
    setup_axes(ax_1, disp_lim, "'child' in 'parent' coordinate system")
    
    ax_2.clear()
    vs.plot_coordinate_system(cs_ref, ax_2, color='r', label="reference")
    vs.plot_coordinate_system(cs_parent, ax_2, color='g', label="parent")
    vs.plot_coordinate_system(cs_child_ref, ax_2, color='y', label="parent + child")
    ax_2.plot([0, p_x], [0, p_y], [0, p_z], 'c--', label="ref -> parent")
    ax_2.plot([p_x, cr_x], [p_y, cr_y], [p_z, cr_z], 'm--', label="parent -> child")
    setup_axes(ax_2, disp_lim * 2, "'parent' and 'child' in reference coordinate system")

    
layout = widgets.Layout(width='200px', height='40px')    
style = {'description_width': 'initial'}

w_po = dict(p_x = FloatSlider(min=-disp_lim, max=disp_lim, step=0.25, description='x', continuous_update=True, layout=layout, style=style),
            p_y = FloatSlider(min=-disp_lim, max=disp_lim, step=0.25, description='y', continuous_update=True, layout=layout, style=style),
            p_z = FloatSlider(min=-disp_lim, max=disp_lim, step=0.25, description='z', continuous_update=True, layout=layout, style=style))

w_pr = dict(p_rx = IntSlider(min=-180, max=180, step=10, description='x', continuous_update=True, layout=layout, style=style),
            p_ry = IntSlider(min=-180, max=180, step=10, description='y', continuous_update=True, layout=layout, style=style),
            p_rz = IntSlider(min=-180, max=180, step=10, description='z', continuous_update=True, layout=layout, style=style))

w_co = dict(c_x = FloatSlider(min=-disp_lim, max=disp_lim, step=0.25, description='x', continuous_update=True, layout=layout, style=style),
            c_y = FloatSlider(min=-disp_lim, max=disp_lim, step=0.25, description='y', continuous_update=True, layout=layout, style=style),
            c_z = FloatSlider(min=-disp_lim, max=disp_lim, step=0.25, description='z', continuous_update=True, layout=layout, style=style))

w_cr = dict(c_rx = IntSlider(min=-180, max=180, step=10, description='x', continuous_update=True, layout=layout, style=style),
            c_ry = IntSlider(min=-180, max=180, step=10, description='y', continuous_update=True, layout=layout, style=style),
            c_rz = IntSlider(min=-180, max=180, step=10, description='z', continuous_update=True, layout=layout, style=style))



w = {**w_po, **w_pr, **w_co, **w_cr}

output = interactive_output(update_output, w)
box_0 = VBox([widgets.Label("parent origin"), *w_po.values()])
box_1 = VBox([widgets.Label("parent rotation (deg)"), *w_pr.values()])
box_2 = VBox([widgets.Label("child origin"), *w_co.values()])
box_3 = VBox([widgets.Label("child rotation (deg)"), *w_cr.values()])
box = HBox([box_0, box_1,box_2, box_3])
display(box)

out
 

Output(layout=Layout(border='2px solid black'))